### importing libraries

In [86]:
from google.cloud import storage
from oauth2client.service_account import ServiceAccountCredentials

import pandas as pd
pd.set_option('display.max_columns', None)

import gspread
gc = gspread.oauth()

import os
import csv
import json
import gzip
import glob

import haversine as hs

from tqdm import tqdm

from datetime import datetime, timedelta

### instantiating the google cloud object

In [11]:
# instantiating the google cloud storage client using the credentials
storage_client = storage.Client.from_service_account_json('ds_credentials_edit.json')

### creating a bucket on google cloud storage

In [3]:
# # declaring the bucket name to be created
# bucket_name = 'ds_competitor_1_marketplace_scraped_data'

# # create a new bucket
# new_bucket = storage_client.bucket(bucket_name)
# new_bucket.storage_class = 'STANDARD'

# # returns Bucket object
# created_bucket = storage_client.create_bucket(new_bucket, location='asia-south1')

### calling the google cloud storage bucket object and pushing not yet uploaded files

In [12]:
# declaring the bucket name from which the date needs to be fetched
bucket_name = 'ds_competitor_1_marketplace_scraped_data'

# get the bucket object from the storage_client
bucket = storage_client.get_bucket(bucket_name)

In [13]:
# declaring empty list to store the filenames that needs to be downloaded from the bucket
blob_file_names = []

# get names of all the blobs in the bucket
for blob_ in bucket.list_blobs():
    blob_file_names.append(blob_.name)

In [14]:
# crawling through folder to get all the file names with local directory
local_path = "/home/customer/Narada/ScrapedData"
all_folders = glob.glob(local_path + '/**')
files_uploaded_dir_remote = []
files_uploaded_dir_local = []

while len(all_folders)!=0:
    local_file = all_folders.pop()
    if not os.path.isfile(local_file):
        all_folders.extend(glob.glob(local_file + '/**'))
    else:
        files_uploaded_dir_remote.append(local_file[22:])
        files_uploaded_dir_local.append(local_file)

In [15]:
# creating dataframe for the directory path and file_names
all_file_dir = pd.DataFrame({'remote':files_uploaded_dir_remote,\
                         'local':files_uploaded_dir_local
                        })

In [16]:
# filtering out the files that need to pushed to google cloud storage
file_dir = all_file_dir[~all_file_dir['remote'].isin(blob_file_names)].reset_index(drop=True)

In [17]:
# count of file to be uploaded to google cloud storage
len(file_dir)

0

In [18]:
# iterating and trying to upload all data programatically
for i in tqdm(range(len(file_dir))):
    # Name of the object to be stored in the bucket
    object_name_in_bucket = bucket.blob(file_dir['remote'].iloc[i])
    
    # Name of the object in local file system
    object_name_in_bucket.upload_from_filename(file_dir['local'].iloc[i])

0it [00:00, ?it/s]


### segregating and labelling the files into "Animal Details", "Seller Details" and "Resource Details"

In [19]:
# segragating "Animal Details", "Resources" and "Seller Info" files
def labelling_files(x):
    # handling bad files
    if len(x.split(".")[0])>5:
        # splitting "file name" into 'sub_file_name_tokens'
        sub_file_name_tokens = x.split(".")[0].split("_")

        # getting 'file_type' - AD/seller/resource
        file_type = sub_file_name_tokens[-1]

        # getting 'date_scraped'
        date_scraped = sub_file_name_tokens[-2]

        # getting 'state' (cross checking)
        state = sub_file_name_tokens[0]
        
        # returns data as tuple
        return [file_type, date_scraped, state]

    else:
        return ['NA', 'NA', 'NA']

In [20]:
# getting file_names to segregate and process files
file_names = [item.split("/")[-1] for item in files_uploaded_dir_local]

In [21]:
# dataFrame with details of all the files in our 'ScrapedData' for 'classified' business
file_df = pd.DataFrame({'file_names':file_names,\
                        'location':files_uploaded_dir_local
                       })

In [22]:
# adding metaData to the 'file_df' DataFrame
file_df[['file_type','date_scraped','state']] = file_df['file_names'].apply(lambda x:labelling_files(x)).tolist()

# dropping 'NA' rows
file_df = file_df[file_df['state']!='NA']

# converting 'date_scraped' to datetime format, and sorting dataFrame
file_df['date_scraped'] = pd.to_datetime(file_df['date_scraped'])
file_df.sort_values(by='date_scraped', inplace=True)
file_df.reset_index(drop=True, inplace=True)

In [23]:
# fetching most recent sample "classified-animal_details" excel file
AD_files = file_df[file_df['file_type']=='AD'].reset_index(drop=True)

# fetching most recent sample "classified-seller" excel file
seller_files = file_df[file_df['file_type']=='seller'].reset_index(drop=True)

# fetching most recent sample "classified-resource" excel file
resource_files = file_df[file_df['file_type']=='resource'].reset_index(drop=True)

### all ANIMAL details files cattle movement analysis

In [24]:
# function to concatenate all "Animal Details" excel files
def get_animal_details(files_df_):
    # declaring empty list to concat all dataFrames
    dflist = []
    
    # looping and concatenating
    for i in range(len(files_df_)):#[-10] to take most recent 10 days scraped data
        
        # file_path of all excel files
        file_path_ = files_df_['location'].iloc[i]

        # reading the file        
        df1 = pd.read_excel(file_path_)
        df1['date_scraped'] = files_df_['date_scraped'].iloc[i]
        df1['date_scraped'] = pd.to_datetime(df1['date_scraped'])
        df1['listing_state'] = files_df_['state'].iloc[i]
        df1['file_age'] = df1['date_scraped'].apply(lambda x: int((datetime.today() - x).days)+1)
        dflist.append(df1)
        
    # concatenating and dropping index
    concatenated_df = pd.concat(dflist).reset_index().drop('index',axis=1)
    
    # renaming columns
    concatenated_df.rename(columns={'_id':'cattle_id','userId':'seller_id', 'seller':'seller_name', 'state':'listing_status'}, inplace=True)
    
    # returning the concatenated dataFrame
    return concatenated_df

In [31]:
# calling the dataFrames concatenating function
all_animal_details_df = get_animal_details(AD_files)

In [39]:
# dropping off "PENDING" and "STA" listing_status - only 2 such instances
all_ad_df = all_animal_details_df[all_animal_details_df['listing_status']=='ACTIVE'].copy()

# getting all unique 'cattle_id'
all_ad_df['cattle_id'] = all_ad_df['cattle_id'].astype(str)

# converting columns into 'datetime' format
all_ad_df['publishedOn'] = pd.to_datetime(all_ad_df['publishedOn'])
all_ad_df['date_scraped'] = pd.to_datetime(all_ad_df['date_scraped'])

# getting the 'last_scraped_date'
last_scraped_date = all_ad_df['date_scraped'].max()

In [40]:
# replacing NaN values
all_ad_df['description'].fillna("-", inplace=True)
all_ad_df['deliveredBefore'].fillna(-999, inplace=True)
all_ad_df['hasDelivered'].fillna(-999, inplace=True)
all_ad_df['isPregnant'].fillna(-999, inplace=True)
all_ad_df['pregnancyMonth'].fillna(-999, inplace=True)
all_ad_df['calf'].fillna('-', inplace=True)
all_ad_df['partnerId'].fillna(-999, inplace=True)
all_ad_df['hasContacted'].fillna(-999, inplace=True)
all_ad_df['breed'].fillna("-", inplace=True)

# replacing int 0 value with str
all_ad_df['calf'].replace(0,'-', inplace=True)
all_ad_df['breed'].replace(0,'-', inplace=True)

# dropping redundant columns
all_ad_df.drop(columns=['description'], inplace=True)
all_ad_df.drop(columns=['district'], inplace=True)
all_ad_df.drop(columns=['dynamicLink'], inplace=True)

In [125]:
# grouped based on "cattle_id", sorted by "date_scraped", taking the last occurence
cattle_sold = pd.DataFrame(all_ad_df.sort_values(by='date_scraped').groupby('cattle_id').tail(1)).reset_index(drop=True)


In [46]:
# function to get listing_status(Sold/Unsold/Fresh_listing) an the no. of days taken to sell item if sold
def get_listing_status(x, last_scraped_dt):
    # "UnSold" listing
    if ((last_scraped_dt-pd.to_datetime(x['date_scraped'])).days)==0:
        days_to_sell = -999
        sold_status = 'UNSOLD'
    # "Fresh" listing
    elif ((last_scraped_dt-pd.to_datetime(x['publishedOn']).tz_localize(None).replace(hour=0, minute=0, second=0, microsecond=0)).days)==0:
        days_to_sell = -999
        sold_status = 'FRESH_LISTING'
    # "De-listed" listing
    elif int((pd.to_datetime(x['date_scraped'])-pd.to_datetime(x['publishedOn']).tz_localize(None).replace(hour=0, minute=0, second=0, microsecond=0)).days)+1==29:
        days_to_sell = -999
        sold_status = 'DELISTED_LISTING'
    # "Sold" listing
    else:
        days_to_sell = int((pd.to_datetime(x['date_scraped'])-pd.to_datetime(x['publishedOn']).tz_localize(None).replace(hour=0, minute=0, second=0, microsecond=0)).days)+1
        sold_status = 'Yes'
     
    # returning the no. of "days_taken_to_sell" and the final "sold_status"
    return [days_to_sell, sold_status]

In [126]:
# applying the "get_listing_status" function
cattle_sold[['days_to_sell', 'sold_status']] = cattle_sold.apply(lambda y:get_listing_status(y, last_scraped_date),axis=1).tolist()

In [127]:
cattle_sold.shape

(95290, 37)

In [128]:
# adding date of results to cattle_sold csv file_name
cattle_sold_file_name = "processed_data_files/cattle_sold_df_"+str(datetime.today().date())+".parquet"

# saving to disk
cattle_sold.to_parquet(cattle_sold_file_name, index=False)

### all SELLER details files contact number mapping

In [52]:
# function to concatenate all "Seller" excel files
def get_seller_details(files_df_):
    # declaring empty list to concat all dataFrames
    dflist = []
    
    # looping and concatenating
    for i in range(len(files_df_)):#[-10] to take most recent 10 days scraped data
        
        # file_path of all excel files
        file_path_ = files_df_['location'].iloc[i]
        
        # reading the file
        df1 = pd.read_excel(file_path_)
        df1['date_scraped'] = files_df_['date_scraped'].iloc[i]
        df1['date_scraped'] = pd.to_datetime(df1['date_scraped'])
        df1['listing_state'] = files_df_['state'].iloc[i]
        df1['file_age'] = df1['date_scraped'].apply(lambda x: int((datetime.today() - x).days)+1)
        dflist.append(df1)
        
    # concatenating and dropping index
    concatenated_df = pd.concat(dflist).reset_index().drop('index',axis=1)
    
    # renaming columns
    concatenated_df.rename(columns={'_id':'cattle_id','userId':'seller_id', 'seller':'seller_name', 'state':'listing_status'}, inplace=True)
    
    # returning the concatenated dataFrame
    return concatenated_df

In [53]:
# calling the dataFrames concatenating function
all_seller_details_df = get_seller_details(seller_files)

In [54]:
# tagging and filtering out 10 digit seller_id
all_seller_details_df['contact_number_y/n'] = all_seller_details_df['id'].apply(lambda x: "Yes" \
                                                                                if(len(str(x))==10) else "No")

In [55]:
all_contacts = all_seller_details_df[all_seller_details_df['contact_number_y/n'] == 'Yes'].reset_index(drop=True).copy()

In [65]:
contacts_cattle_id = all_contacts[['cattle_id','id']].copy()
contacts_cattle_id.rename(columns={'id':'contact_number'}, inplace=True)
contacts_cattle_id['contact_number'] = contacts_cattle_id['contact_number'].astype(int)
contacts_cattle_id.drop_duplicates(inplace=True)
contacts_cattle_id.reset_index(drop=True, inplace=True)

In [153]:
contacts_cattle_id.shape

(22696, 2)

In [160]:
# adding date of results to all_animal_details csv file_name
contacts_cattle_id_file_name = "processed_data_files/all_seller_details_df_"+str(datetime.today().date())+".parquet"

# saving to disk
contacts_cattle_id.to_parquet(contacts_cattle_id_file_name, index=False)

### all cattles pincode mapping - gmap_lat_long

In [77]:
# reading the current gmap_lat_long_mapping csv
gmap_lat_long_mapping_df = pd.read_csv("gmap_mapping_result.csv")

In [87]:
# Written to "classified_AD_movement_analysis"
worksheet = gc.open_by_key('1yybUEyelVjqX4AfwRqgRJ4EK19CakSNVaFHkCqzZ8PA').worksheet('current')
# raw_data
lat_long_data = [gmap_lat_long_mapping_df.columns.astype(str).tolist()]+gmap_lat_long_mapping_df.values.astype(str).tolist()
# writing the sheet
worksheet.update('A1', lat_long_data)

{'spreadsheetId': '1yybUEyelVjqX4AfwRqgRJ4EK19CakSNVaFHkCqzZ8PA',
 'updatedRange': 'current!A1:K91873',
 'updatedRows': 91873,
 'updatedColumns': 11,
 'updatedCells': 1010603}

In [81]:
# filtering out 'cattle_ids' that need to be mapped
tbd_df = cattle_sold[~cattle_sold['cattle_id'].isin(gmap_lat_long_mapping_df['cattle_id'].tolist())].reset_index(drop=True)


In [82]:
tbd_df.shape

(3418, 37)

In [88]:
# Written to "classified_AD_movement_analysis"
worksheet = gc.open_by_key('1yybUEyelVjqX4AfwRqgRJ4EK19CakSNVaFHkCqzZ8PA').worksheet('tbd_cattles')
# raw_data
tbd_cattles_data = [tbd_df.columns.astype(str).tolist()]+tbd_df.values.astype(str).tolist()
# writing the sheet
worksheet.update('A1', tbd_cattles_data)

{'spreadsheetId': '1yybUEyelVjqX4AfwRqgRJ4EK19CakSNVaFHkCqzZ8PA',
 'updatedRange': 'tbd_cattles!A1:AK3419',
 'updatedRows': 3419,
 'updatedColumns': 37,
 'updatedCells': 126503}

In [83]:
# reading the GMAP lat_long data
lat_long_df = pd.read_csv("GMAP_full.csv")

In [ ]:
for i, row in tqdm(tbd_df.iterrows()):
    lat = float(row['lat'])
    long = float(row['long'])
    source = (lat,long)
    distance_matrixs = {}
    for j, row_ in lat_long_df.iterrows():
        glat = float(row_['Lat_GMAP'])
        glong = float(row_['Long_GMAP'])
        destination = (glat,glong)
        distance = hs.haversine(source,destination)
        distance_matrixs[j] = distance

    distance_matrixs = sorted(distance_matrixs.items(), key=lambda x:x[1])
    distance_mats = dict(distance_matrixs)
    max_idx = list(distance_mats.keys())[0]

    tbd_df.loc[i,'idx'] = max_idx
    tbd_df.loc[i,'GLong'] = lat_long_df.iloc[max_idx]['Long_GMAP']
    tbd_df.loc[i,'GLat'] = lat_long_df.iloc[max_idx]['Lat_GMAP']
    tbd_df.loc[i,'Pincode'] = lat_long_df.iloc[max_idx]['Pincode']
    tbd_df.loc[i,'Distance'] = distance_mats[max_idx]*1.2

In [84]:
# columns to be retained from tbd_df
col_names = gmap_lat_long_mapping_df.columns.tolist()

In [ ]:
# concatenated "gmap_mapping_result" dataframe
gmap_mapping_result = pd.concat([gmap_lat_long_mapping_df,tbd_df[col_names]],axis=0).reset_index(drop=True)

In [93]:
gmap_lat_long_mapping_df.shape

(91872, 11)

In [90]:
# adding date of results to all_animal_details csv file_name
gmap_lat_long_file_name = "processed_data_files/gmap_lat_long_mapping_df_"+str(datetime.today().date())+".parquet"

# saving to disk
gmap_lat_long_mapping_df.to_parquet(gmap_lat_long_file_name, index=False)

In [242]:
# saving to disk
gmap_lat_long_mapping_df.to_csv("gmap_mapping_result.csv", index=False)

### indian postal code pincode.csv

In [181]:
# reading csv
pincode_df = pd.read_csv("pincode.csv", low_memory=False)

In [182]:
pincode_df.shape

(157126, 11)

In [183]:
pincodes_df = pincode_df[['Pincode','District','StateName']].copy()

In [184]:
pincodes_df = pincodes_df.rename(columns={'Pincode':'gmap_pincode',\
                            'District':'gmap_district',\
                            'StateName':'gmap_state',\
                           })

In [185]:
pincodes_df = pincodes_df.drop_duplicates(subset=['gmap_pincode']).reset_index(drop=True)

In [186]:
pincodes_df.shape

(19300, 3)

### adding features for reports/analysis

#### contacts mapping

In [161]:
cattle_sold.shape

(95290, 37)

In [164]:
contacts_cattle_id.shape

(22696, 2)

In [166]:
cattle_sold = cattle_sold.merge(contacts_cattle_id, left_on='cattle_id', right_on='cattle_id', how='left').fillna(-999)

In [167]:
cattle_sold['contact_number'] = cattle_sold['contact_number'].astype(int)

In [168]:
cattle_sold.shape

(95290, 38)

#### gmap lat long mapping

In [169]:
gmap_lat_long_df = gmap_lat_long_mapping_df[['cattle_id','GLong','GLat','Pincode']].copy()

In [170]:
gmap_lat_long_df = gmap_lat_long_df.rename(columns={'Pincode':'gmap_pincode',\
                                 'GLong':'gmap_long',\
                                 'GLat':'gmap_lat',\
                                })

In [171]:
cattle_sold = cattle_sold.merge(gmap_lat_long_df, left_on='cattle_id', right_on='cattle_id', how='left').fillna(-999)

In [172]:
cattle_sold['gmap_pincode'] = cattle_sold['gmap_pincode'].astype(int)

In [173]:
cattle_sold.shape

(95290, 41)

#### pincode state and district mapping

In [187]:
cattle_sold.shape

(95290, 41)

In [188]:
cattle_sold = cattle_sold.merge(pincodes_df, left_on='gmap_pincode', right_on='gmap_pincode', how='left').fillna('-')

In [189]:
cattle_sold = cattle_sold.reset_index(drop=True)

In [190]:
cattle_sold.shape

(95290, 43)

#### adding week names and date features

In [191]:
cattle_sold.shape

(95290, 43)

In [227]:
cattle_sold['month_of_sale'] = cattle_sold['date_scraped'].dt.month_name()
cattle_sold['week_of_sale'] = (cattle_sold['date_scraped']+pd.DateOffset(1)).dt.isocalendar().week
cattle_sold['year_of_sale'] = cattle_sold['date_scraped'].dt.year
cattle_sold['weekday'] = cattle_sold['date_scraped'].dt.strftime('%A')
cattle_sold['week_start_date'] = cattle_sold['date_scraped'].apply(lambda x: (x-timedelta(days=(x.weekday()+1)%7)).strftime('%d %b'))
cattle_sold['week_end_date'] = cattle_sold['date_scraped'].apply(lambda x: (x-timedelta(days=(x.weekday()+1)%7) + timedelta(days=6)).strftime('%d %b'))
cattle_sold['sale_week'] = cattle_sold.apply(lambda x: str(x['week_start_date']) + " - " + str(x['week_end_date']), axis=1)
cattle_sold['sale_day'] = cattle_sold['date_scraped'].apply(lambda x: str(x.strftime('%d %b')) + ": " + str(x.strftime('%a')))


In [228]:
cattle_sold.shape

(95290, 51)

#### renaming and standardizing the column names

In [230]:
# making a copy of the dataframe
data = cattle_sold.copy()

In [232]:
# standardizing column names for google cloud storage
col_names_mapping = {'viewCount':'view_count_animall',\
                     'callCount':'call_count_animall',\
                     '_score':'score',\
                     'long':'listing_long',\
                     'lat':'listing_lat',\
                     'highestMilk':'highest_milk',\
                     'locationName':'location_name',\
                     'isNegotiable':'is_negotiable',\
                     'publishedOn':'published_on',\
                     'currentMilk':'current_milk',\
                     'deliveredBefore':'delivered_before',\
                     'hasDelivered':'has_delivered',\
                     'isPregnant':'is_pregnant',\
                     'pregnancyMonth':'pregnancy_month',\
                     'age':'cattle_age',\
                     'animalType':'animal_type',\
                     'partnerId':'partner_id',\
                     'hasContacted':'has_contacted',\
                    }

In [233]:
# renaming columns
data = data.rename(columns=col_names_mapping)

#### standardizing the column data types

In [235]:
data['distance'] = data['distance'].astype(float)
data['rating'] = data['rating'].astype(float)
data['view_count_animall'] = data['view_count_animall'].astype(int)
data['call_count_animall'] = data['call_count_animall'].astype(int)
data['recency'] = data['recency'].astype(int)
data['score'] = data['score'].astype(float)
data['listing_long'] = data['listing_long'].astype(float)
data['listing_lat'] = data['listing_lat'].astype(float)
data['highest_milk'] = data['highest_milk'].astype(float)
data['is_negotiable'] = data['is_negotiable'].astype(int)
data['published_on'] = pd.to_datetime(data['published_on'])
data['current_milk'] = data['current_milk'].astype(float)
data['delivered_before'] = data['delivered_before'].apply(lambda x: float(x) if x!='-' else x)
data['has_delivered'] = data['has_delivered'].apply(lambda x: float(x) if x!='-' else x)
data['is_pregnant'] = data['is_pregnant'].apply(lambda x: float(x) if x!='-' else x)
data['pregnancy_month'] = data['pregnancy_month'].apply(lambda x: float(x) if x!='-' else x)
data['price'] = data['price'].astype(float)
data['cattle_age'] = data['cattle_age'].astype(float)
data['lactation'] = data['lactation'].astype(int)
data['date_scraped'] = pd.to_datetime(data['date_scraped'])
data['file_age'] = data['file_age'].astype(int)
data['days_to_sell'] = data['days_to_sell'].apply(lambda x: int(x) if len(str(x))<=2 else x)
data['week_of_sale'] = data['week_of_sale'].apply(lambda x: int(x) if x!='UNSOLD' else x)
data['year_of_sale'] = data['year_of_sale'].apply(lambda x: int(x) if x!='UNSOLD' else x)
data['gmap_pincode'] = data['gmap_pincode'].apply(lambda x: int(x) if x!='-' else x)
data['contact_number'] = data['contact_number'].apply(lambda x: int(x) if x not in ['NA','-'] else x)

### converting to parquet

In [236]:
data.to_parquet('furnished_data/cattle_sold_data.parquet', index=False)

### creating a bucket on google cloud storage for final furnished data

In [237]:
# # declaring the bucket name to be created
# bucket_name = 'ds_competitor_1_marketplace_reporting'

# # create a new bucket
# new_bucket = storage_client.bucket(bucket_name)
# new_bucket.storage_class = 'STANDARD'

# # returns Bucket object
# created_bucket = storage_client.create_bucket(new_bucket, location='asia-south1')

### pushing to google cloud storage bucket

In [238]:
# declaring the bucket name from which the date needs to be fetched
bucket_name = 'ds_competitor_1_marketplace_reporting'

# get the bucket object from the storage_client
bucket = storage_client.get_bucket(bucket_name)

In [239]:
# Name of the object to be stored in the bucket
object_name_in_bucket = bucket.blob('movement_analysis.parquet')

# local folder name
local_folder = os.getcwd()

# local file location
filename = "%s/%s" % (local_folder, 'furnished_data/cattle_sold_data.parquet')

# Name of the object in local file system
object_name_in_bucket.upload_from_filename(filename)

### reading parquet file

In [240]:
movement_analysis_df = pd.read_parquet('furnished_data/cattle_sold_data.parquet', engine='fastparquet')

In [241]:
movement_analysis_df.head()

,cattle_id,seller_id,distance,rating,view_count_animall,call_count_animall,recency,score,listing_long,listing_lat,gender,listing_status,highest_milk,seller_name,location_name,is_negotiable,published_on,current_milk,delivered_before,has_delivered,is_pregnant,pregnancy_month,price,cattle_age,animal_type,breed,calf,lactation,partner_id,has_contacted,type,coordinates,date_scraped,listing_state,file_age,days_to_sell,sold_status,contact_number,gmap_long,gmap_lat,gmap_pincode,gmap_district,gmap_state,month_of_sale,week_of_sale,year_of_sale,weekday,week_start_date,week_end_date,sale_week,sale_day
0,62db79e6d17eaf000bd8235c,15504404057,22593.000000,3.0,74,3,0,94.020466,77.4294,28.6380,FEMALE,ACTIVE,14.0,Rajendra Singh,"Shanti Nagar, Nai Basti Dundahera, Ghaziabad, ...",1,2022-07-24 11:00:41.809000+00:00,10.0,0.00,1.0,0.0,-999.0,38000.0,5.0,COW,AMERICAN CROSS,MALE,2,-999.0,-999.0,0,0,2022-07-25,DELHI,60,2,Yes,-999,77.4349,28.6302,201016,GHAZIABAD,UTTAR PRADESH,July,30,2022,Monday,24 Jul,30 Jul,24 Jul - 30 Jul,25 Jul: Mon
1,62dc1cfa136fa3000b12e148,9971568876,6734.546414,4.0,204,28,1,94.302968,77.3570,28.6908,FEMALE,ACTIVE,15.0,aasim,"ग़ाज़ियाबाद, ग़ाज़ियाबाद ज़िला, उत्तर प्रदेश 2...",1,2022-07-24 01:50:32.946000+00:00,12.0,0.25,1.0,0.0,-999.0,86000.0,0.0,BUFFALO,MURRAH CROSS,-,2,-999.0,-999.0,0,0,2022-07-25,DELHI,60,2,Yes,9971568876,77.3748,28.6806,201007,GHAZIABAD,UTTAR PRADESH,July,30,2022,Monday,24 Jul,30 Jul,24 Jul - 30 Jul,25 Jul: Mon
2,62d90613cc5dfe000b998567,15503320123,13008.000000,4.0,193,35,3,94.434781,77.4128,28.7451,FEMALE,ACTIVE,15.0,Kasama,"Ghaziabad, Ghaziabad District, Uttar Pradesh, ...",1,2022-07-21 07:56:39.855000+00:00,8.0,4.00,1.0,1.0,1.0,15000.0,0.0,COW,JERSEY CROSS,FEMALE,2,-999.0,-999.0,0,0,2022-07-25,DELHI,60,5,Yes,-999,77.4128,28.7451,201003,GHAZIABAD,UTTAR PRADESH,July,30,2022,Monday,24 Jul,30 Jul,24 Jul - 30 Jul,25 Jul: Mon
3,62dd1d9086feac000aea6332,15506020485,14064.189741,3.0,83,2,0,93.601063,77.3133,28.6504,FEMALE,ACTIVE,11.0,krishansharma,"Anand Vihar, Vivek Vihar Tehsil, Shahdara, Del...",1,2022-07-24 13:07:01.312000+00:00,0.0,-999.00,0.0,1.0,2.0,11500.0,2.0,COW,DESI CROSS,-,2,-999.0,-999.0,0,0,2022-07-25,DELHI,60,2,Yes,-999,77.3027,28.6502,110092,EAST,DELHI,July,30,2022,Monday,24 Jul,30 Jul,24 Jul - 30 Jul,25 Jul: Mon
4,62dd29eb07b924000aeb4502,15505988041,26287.119752,3.0,16,0,0,94.551751,77.4460,28.5185,FEMALE,ACTIVE,14.0,Navneet,"ग्रेटर नोएडा, Gautam Buddha Nagar, Gautam Budd...",1,2022-07-25 03:52:28.101000+00:00,0.0,-999.00,0.0,1.0,9.0,92000.0,5.0,BUFFALO,MURRAH CROSS,-,2,-999.0,-999.0,0,0,2022-07-25,DELHI,60,1,Yes,-999,77.4521,28.5239,201306,GAUTAM BUDDHA NAGAR,UTTAR PRADESH,July,30,2022,Monday,24 Jul,30 Jul,24 Jul - 30 Jul,25 Jul: Mon
